In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# export
import pandas as pd
import numpy as np
from datetime import timedelta

In [3]:
from kaggle_1c_predict_future_sales.trivial_predict import read_train, forecast_last_average

train_all_time = read_train()
train_all_time.head(2)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-02-01,0,59,22154,999.0,1.0
1,2013-03-01,0,25,2552,899.0,1.0


In [4]:
train_all_time.agg({'date': ['min', 'max']})

,date
min,2013-01-01
max,2015-12-10


In [63]:
test = pd.read_csv('raw/test.csv')

def single_fold(train_all_time, forecast_start, forecast_length_days):
    id_cols = ['item_id', 'shop_id']

    forecast_target_raw = train_all_time.query('date >= @forecast_start and date <= @forecast_end')
    forecast_target_mo = (forecast_target_raw
        .groupby(id_cols, as_index=False)
        .agg({'item_cnt_day': 'sum'})
        .rename({'item_cnt_day': 'item_cnt_month'}, axis=1)
    )

    forecast_target_mo = (test
                          .merge(forecast_target_mo, on=id_cols, how='left')
                          .fillna(0)
                         )

    train_start = train_all_time.query('date < @forecast_start')

    predicted_mo = forecast_last_average(
        train_start,
        forecast_month_length=30,
        train_interval_len=30,
        target_forecast_start=forecast_start
    )

    predicted_mo['item_cnt_month'] = predicted_mo['item_cnt_month'].clip(0, 20)
    forecast_target_mo['item_cnt_month'] = forecast_target_mo['item_cnt_month'].clip(0, 20)
    from sklearn.metrics import mean_squared_error 
    diff = (forecast_target_mo
        .merge(predicted_mo, on=id_cols, suffixes=('_target', '_predicted'))
    )
    rmse = mean_squared_error(diff['item_cnt_month_target'], diff['item_cnt_month_predicted'])
    return rmse

single_fold(train_all_time,
           forecast_start=pd.to_datetime('2014-06-01'),
           forecast_length_days=30)

5.664616599898437

In [79]:
target_forecast_start = pd.to_datetime('2015-11-01')
forecast_length_days = 30

fold_rmses = []
fold_forecast_starts = []
n_folds = 10
fold_shift_days = 30
for delta in range(1, 1+n_folds):
    forecast_start = target_forecast_start - timedelta(days=delta*fold_shift_days)
    rmse = single_fold(train_all_time,
           forecast_start=forecast_start,
           forecast_length_days=30)
    fold_rmses.append(rmse)
    fold_forecast_starts.append(forecast_start)

In [80]:
fold_rmses

[7.832448441506845,
 6.696074651037248,
 8.794150731158606,
 9.206318029298632,
 8.553117826365108,
 17.32010097822657,
 11.44927746363153,
 11.494969613400746,
 9.83613969441846,
 7.348355858252634]

In [81]:
# 33.44 doesn't square well with LB of 8 or like 11
# 7.26 after clipping is mucho closero
# 10 folds x 20 days was closer (8.02) and faster than 20 folds x 10 days (7.97)
sum(fold_rmses) / len(fold_rmses)

9.853095328729637